roberta_lstm_ex1验证集错例分析

In [1]:
from pathlib import Path

In [2]:
import sys
sys.path.insert(0, '../')

In [3]:
from metrics import get_entities
from utils import IO2STR

In [4]:
val_data_dir = Path('../data/val/')

In [5]:
with open(val_data_dir/'sentences.txt', 'r', encoding='utf-8') as f_sen,\
    open(val_data_dir/'tags.txt', 'r', encoding='utf-8') as f_tag:
    sentences = [line.strip().split(' ') for line in f_sen]
    tags = [line.strip().split(' ') for line in f_tag]

获取gold label

In [6]:
y_true = []
for s,t in zip(sentences, tags):
    entities = get_entities(t)
    sample = []
    for entity in entities:
        entity_con = s[entity[1]:entity[2]+1]
        entity_type = IO2STR[entity[0]]
        sample.append((entity_type, ''.join(entity_con)))
    y_true.append(sample) 

获取pre label

In [7]:
ex1_dir = Path('../experiments/ex1')

In [8]:
import json

In [9]:
with open(ex1_dir/'submit_val.txt', 'r', encoding='utf-8') as f:
    pre = [json.loads(line.strip()) for line in f]

In [10]:
y_pre = []
for p in pre:
    text = p['originalText']
    entities = p['entities']
    sample = []
    for entity in entities:
        entity_con = text[entity['start_pos']:entity['end_pos']]
        sample.append((entity['label_type'], entity_con))
    y_pre.append(sample)

观察结果

In [11]:
for sen, true, pre in zip(sentences, y_true, y_pre):
    print(''.join(sen))
    print('-*-'*40)
    # 真实标签里有而预测结果里没有的（影响Recall）
    print(list(set(true).difference(set(pre))))
    print('-*-'*40)
    # 预测结果里有而真实标签里没有的（影响Precision）
    print(list(set(pre).difference(set(true))))
    print('-*-'*40)
    print(true)
    print('-*-'*40)
    print(pre)
    print('\n')

再次入院病程记录患儿半月前无明显诱因出现咳嗽，呈阵发性单咳，夜间明显，一日十余次，咳嗽剧烈时可有呕吐，呕吐物为胃内容物及痰，痰为粘白色，患儿自己不会咳痰。间隔1日出现发热，体温39℃左右，热型不规则，吃美林后温度可降至正常，夜间发热较明显，患儿**我院门诊反复就诊，诊断为呼吸道感染，分别给予头孢噻肟输液２天，头孢美唑输液２天，阿奇霉素输液２天，及对症处理，患儿发热不退，咳嗽未见好转，于4.03收入院给予静脉输注头孢美唑、阿奇霉素及口服克拉霉素“等药物治疗7天，患儿**咳嗽，体温正常后出院，出院后患儿仍有咳嗽，今晨起出现发热，最高体温38.5℃，伴有流涕、鼻塞，咳嗽加重，故来我院复诊，门诊拟肺炎再次收入院。发病以来无寒战，惊厥，无头痛，无喷射样呕吐，无腹泻。发病期间精神尚可，反应好，食欲下降，入眠尚可，二便正常。首次病程记录患儿５日前无明显诱因出现咳嗽，呈阵发性单咳，夜间明显，一日十余次，咳嗽剧烈时可有呕吐，呕吐物为胃内容物及痰，痰为粘白色，患儿自己不会咳痰。３日前发热，体温３９℃左右，热型不规则，吃美林后温度可降至正常，夜间发热较明显，无寒战，惊厥。无头痛，无喷射样呕吐，无腹泻。患儿**我院门诊反复就诊，诊断为呼吸道感染，分别给予头孢噻肟输液２天，头孢美唑输液２天，阿奇霉素输液２天，及对症处理，患儿发热不退，咳嗽未见好转，故于今日再次来我院就诊，门诊以支气管肺炎为诊断收入院。患儿发病期间精神好，反应好，食欲下降，入眠尚可，二便正常。
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('药物', '吃美林'), ('解剖部位', '间隔')]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('药物', '美林')]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-

获取词表

In [31]:
vocab_dir = '../data/vocab/ne_vocab.txt'

In [32]:
def get_vocab():
    """获取词表"""
    with open(vocab_dir, 'r', encoding='utf-8') as f_vocab:
        vocab = [line.strip().split('\t') for line in f_vocab]

    vocab_dict = {
        '疾病和诊断': [en[0] for en in vocab if en[1] == '疾病和诊断'],
        '影像检查': [en[0] for en in vocab if en[1] == '影像检查'],
        '实验室检验': [en[0] for en in vocab if en[1] == '实验室检验'],
        '手术': [en[0] for en in vocab if en[1] == '手术'],
        '药物': [en[0] for en in vocab if en[1] == '药物'],
        '解剖部位': [en[0] for en in vocab if en[1] == '解剖部位']
    }
    return vocab_dict

In [33]:
vocab = get_vocab()

In [35]:
vocab['疾病和诊断']

['肝右叶占位病变',
 '宫颈鳞癌（IB1期，II级）术后',
 '高血压病3级极高危组',
 '高血压心脏病',
 '肝脏原发性巨块型中分化肝细胞癌',
 '胸9椎体及附件骨质破坏、转移',
 '肾性高血',
 '（胃大弯、小弯）溃疡型中分化管状腺癌',
 '（乙状结肠）腺癌',
 '浅表性胃窦炎',
 '卵巢腺癌',
 '宫颈鳞癌1B2期',
 '子宫内膜腺癌IIIC期（G3）',
 '手足综合症',
 '子宫内膜非典型增生',
 '盆腔巨内巨大囊性包块',
 '胃病',
 '肝包膜转移',
 '（隆突上淋巴结）可见低分化癌转移',
 '胃窦低分化腺癌',
 '卵巢颗粒细胞瘤',
 'CINIII',
 '两肺多发占位',
 '急性肾损伤',
 '糖尿病',
 '(食管)，：食管溃疡型鳞状细胞癌II级',
 '管状腺癌',
 '高血压2级很高危',
 'HCC',
 '左肾积水',
 '左半肝肝脏原发性巨块型中分化肝癌',
 '肝左后缘斑点状钙化',
 '食管髓质型高分化鳞状细胞癌',
 '中肾管癌',
 '胃下垂',
 '盲肠癌术后',
 '胸椎转移',
 '肺部转移',
 '肠腺癌转移',
 '结肠肿瘤',
 '肝内胆管癌',
 '直肠癌术后吻合口复发',
 '胆管炎',
 '黏膜上子宫肌瘤',
 '低分化腺癌（大部分胃粘液腺癌）',
 '直肠黑色素瘤',
 '结肠脾曲癌',
 '残胃异时性原发癌',
 '卵巢子宫内膜样腺癌(II期，I级)术后',
 '乙状结肠溃疡型中分化管状乳头状腺癌',
 '（左半）结肠溃疡型低分化腺癌',
 '根治性全胃切除术后',
 '食管胃交界溃疡型管状腺癌II',
 '慢性浅表性胃炎伴糜烂、胆汁返流',
 '局限性硬皮病',
 '胃角溃疡(S2期)',
 '甲状腺功能低上II度',
 '胃肠息肉',
 '食管胃交界处溃疡型中－低分化腺癌',
 '食管胃底静脉曲张',
 '（右乳）分泌粘液癌（印戒细胞癌）',
 'ST段改变',
 '慢性肝病性胆囊',
 '恶性肿瘤术后化疗',
 '慢性肾功能不全',
 '肠隆起型绒毛状-管状腺癌II级',
 '红细胞再生障碍性贫血',
 '胃窦术后化疗',
 '左半结肠回盲部隆起型管状腺癌II级',
 '进展期直肠癌，粘液腺瘤及中分化管状腺癌',
 '慢性发炎',
 '上肢动脉硬化',
 '心

In [40]:
import numpy as np

In [59]:
y_pre_vocab = []
for pre in y_pre:
    sample=[]
    for ne in pre:
        match_voc = []
        if ne[1] in vocab[ne[0]]:
            continue
        elif len(ne[1]) > 15:
            for voc in vocab[ne[0]]:
                if voc in ne[1]:
                    match_voc.append(voc)
            if len(match_voc) != 0:
                min_idx = np.argmin([len(ne[1])-len(mv) for mv in match_voc])
                mv = match_voc[min_idx]
                sample.append((ne,mv))
    y_pre_vocab.append(sample)

In [60]:
y_pre_vocab

[[],
 [],
 [],
 [(('疾病和诊断', '食管胃交界腺癌化疗后）食管及胃交界部低分化腺癌'), '食管胃交界腺癌'),
  (('疾病和诊断', '（食管胃交界腺癌化疗后）食管及胃交界部低分化腺癌'), '食管胃交界腺癌')],
 [],
 [],
 [],
 [],
 [(('疾病和诊断', '食管胃交界溃疡型管状腺癌，II-III级'), '管状腺癌')],
 [],
 [],
 [(('疾病和诊断', '（胃）中分化腺癌伴粘液腺癌（后者约占15%），溃疡型'), '（胃）中分化腺癌')],
 [],
 [(('手术', '根治性全胃切除术+食管空肠ROUX-Y吻合术'), '根治性全胃切除术')],
 [(('疾病和诊断', '进展期直肠癌：浸润溃疡型，中分化管状腺癌'), '管状腺癌')],
 [(('手术', '经腹广泛子宫+两附件切除+盆腔淋巴清扫术'), '盆腔淋巴清扫')],
 [(('疾病和诊断', '贲门癌伴肝脏、盆底种植、多发淋巴结转移可能'), '贲门癌伴肝脏、盆底种植、多发淋巴结转移')],
 [],
 [],
 [(('疾病和诊断', '（左半结肠）中低分化腺癌，溃疡型'), '（左半结肠）中低分化腺癌')],
 [],
 [],
 [(('手术', '腹腔镜上直肠癌根治术（MILES）+放射性粒子植入术'), '腹腔镜上直肠癌根治术（MILES）')],
 [],
 [],
 [],
 [(('疾病和诊断', '溃疡型低分化腺癌（大部分为粘液腺癌'), '溃疡型低分化腺癌')],
 [(('疾病和诊断', '胃体胃窦癌(CT4N2M0,IIIB期)'), '胃体胃窦癌'),
  (('疾病和诊断', '胃中至低分化管状腺癌（LAUREN，分型：肠型）'), '低分化管状腺癌'),
  (('疾病和诊断', '胃中至低分化管状腺癌（LAUREN，'), '低分化管状腺癌')],
 [],
 [],
 [(('手术', '腹腔镜上广泛性子宫切除术+两侧附件切除+盆腔淋巴结清扫术+盆腔粘连松解术'), '盆腔淋巴结清扫术')],
 [],
 [(('疾病和诊断', '直肠癌综合治疗后肝、脾、腹膜等多发转移✄KRASG13D突变型'), '直肠癌')],
 [(('疾病和诊断', '溃疡型低分化腺癌，伴印戒细胞癌，LAUREN，

In [57]:
for true, pre in zip(y_true, y_pre):
    # 真实标签里有而预测结果里没有的（影响Recall）
    print(list(set(true).difference(set(pre))))
    print('-*-'*40)
    # 预测结果里有而真实标签里没有的（影响Precision）
    print(list(set(pre).difference(set(true))))
    print('-*-'*40)
    print(true)
    print('-*-'*40)
    print(pre)
    print('\n')

[('解剖部位', '间隔'), ('药物', '吃美林')]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('药物', '美林')]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('解剖部位', '胃'), ('解剖部位', '间隔'), ('药物', '吃美林'), ('疾病和诊断', '呼吸道感染'), ('药物', '头孢噻肟'), ('药物', '头孢美唑'), ('药物', '阿奇霉素'), ('药物', '头孢美唑'), ('药物', '阿奇霉素'), ('药物', '克拉霉素'), ('解剖部位', '鼻'), ('疾病和诊断', '肺炎'), ('解剖部位', '头'), ('解剖部位', '腹'), ('解剖部位', '胃'), ('药物', '吃美林'), ('解剖部位', '头'), ('解剖部位', '腹'), ('疾病和诊断', '呼吸道感染'), ('药物', '头孢噻肟'), ('药物', '头孢美唑'), ('药物', '阿奇霉素'), ('疾病和诊断', '支气管肺炎')]
-*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*--*-
[('解剖部位', '头'), ('解剖部位', '腹'), ('疾病和诊断', '呼吸道感染'), ('疾病和诊断', '支气管肺炎'), ('疾病和诊断', '呼吸道感染'), ('药物', '头孢美唑'), ('药物', '克拉霉素'), ('药物', '头孢噻肟'), ('药物', '阿奇霉素'), ('解剖部位', '鼻'), ('药物', '头孢噻肟'), ('疾病和诊断', '肺炎'), ('药物', '美林'), 

对比2019data和unlabel data

In [3]:
from pathlib import Path
import json

In [6]:
data_dir = Path('../data/pseudo/')

In [8]:
with open(data_dir/'2019data.txt', 'r', encoding='utf-8') as f_2019:
     data_2019 = [json.loads(line.strip())['originalText'] for line in f_2019]

In [10]:
with open(data_dir/'unlabel_data.txt', 'r', encoding='utf-8') as f_un:
    data_un = [line.strip() for line in f_un]

In [13]:
for l1 in data_2019:
    for l2 in data_un:
        if l1==l2:
            print('yes')

In [15]:
text_data_dir = Path('../ccks2_task1_val/')

In [16]:
with open(text_data_dir/'task1_no_val.txt', 'r', encoding='gbk') as f_test:
    data_test = [json.loads(line.strip())['originalText'] for line in f_test]

In [21]:
count = 0
for l1 in data_2019:
    for l2 in data_test:
        if l1==l2:
            count += 1
            print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
